In [ ]:
# Installing the necessary libraries
!pip install transformers==4.20.1
!pip install pandas==1.3.5
!pip install numpy==1.21.5
!pip install scipy==1.7.3

In [1]:
# Importing necessary modules
from transformers import AutoModelForMaskedLM, AutoTokenizer
import pandas as pd
import numpy as np
from scipy.special import softmax

In [2]:
# Defining the model name
model_name = "bert-base-cased"

# Loading the pre-trained model and tokenizer
model = AutoModelForMaskedLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [3]:
# Defining the mask token
mask = tokenizer.mask_token

# Defining the sentence
sentence = f"I want to {mask} pizza for tonight."

# Tokenizing the sentence
tokens = tokenizer.tokenize(sentence)
tokens

['I', 'want', 'to', '[MASK]', 'pizza', 'for', 'tonight', '.']

In [4]:
# Encoding the input sentence and getting model predictions
encoded_inputs = tokenizer(sentence, return_tensors="pt")
output = model(**encoded_inputs)
output

MaskedLMOutput(loss=None, logits=tensor([[[ -7.3723,  -7.2489,  -7.4421,  ...,  -6.3119,  -5.9369,  -6.4257],
         [ -7.9311,  -8.2282,  -8.0326,  ...,  -6.7387,  -6.4877,  -6.9525],
         [-12.0500, -11.7972, -12.5776,  ...,  -8.4518,  -6.7310,  -8.2586],
         ...,
         [-10.2204, -10.4315,  -9.9993,  ...,  -7.9570,  -6.7194,  -9.3618],
         [-12.4471, -12.5367, -12.5614,  ...,  -9.9086,  -9.4219, -11.1770],
         [-14.3657, -14.5227, -15.0017,  ..., -11.9715, -11.6569, -13.4498]]],
       grad_fn=<AddBackward0>), hidden_states=None, attentions=None)

In [5]:
# Detaching the logits from the model output and converting to numpy array
logits = output.logits.detach().numpy()[0]
logits

array([[ -7.372293 ,  -7.2488623,  -7.4421434, ...,  -6.311862 ,
         -5.936892 ,  -6.425681 ],
       [ -7.931119 ,  -8.2282095,  -8.03259  , ...,  -6.7387447,
         -6.487723 ,  -6.952524 ],
       [-12.050009 , -11.797212 , -12.577611 , ...,  -8.451774 ,
         -6.731021 ,  -8.258568 ],
       ...,
       [-10.22041  , -10.431484 ,  -9.99926  , ...,  -7.956993 ,
         -6.719401 ,  -9.361792 ],
       [-12.447123 , -12.536707 , -12.561405 , ...,  -9.908551 ,
         -9.421909 , -11.176951 ],
       [-14.365712 , -14.5227165, -15.001677 , ..., -11.971548 ,
        -11.656921 , -13.449786 ]], dtype=float32)

In [6]:
# Extracting the logits for the masked token and calculating the confidence scores
masked_logits = logits[tokens.index(mask) + 1]
confidence_scores = softmax(masked_logits)
confidence_scores

array([2.9159811e-10, 4.0784948e-10, 5.2927940e-10, ..., 8.4446100e-10,
       6.2026295e-09, 1.6282753e-09], dtype=float32)

In [7]:
# Iterating over the top 5 predicted tokens and printing the sentences with the masked token replaced
for i in np.argsort(confidence_scores)[::-1][:5]:
    pred_token = tokenizer.decode(i)
    score = confidence_scores[i]

    # print(pred_token, score)
    print(sentence.replace(mask, pred_token))

2024-03-05 22:44:40.291725: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory


I want to have pizza for tonight.
I want to get pizza for tonight.
I want to eat pizza for tonight.
I want to make pizza for tonight.
I want to order pizza for tonight.
